# Day 19

## Part 1

The Elves of Gear Island are thankful for your help and send you on your way. They even have a hang glider that someone stole from Desert Island; since you're already going that direction, it would help them a lot if you would use it to get down there and return it to them.

As you reach the bottom of the relentless avalanche of machine parts, you discover that they're already forming a formidable heap. Don't worry, though - a group of Elves is already here organizing the parts, and they have a system.

To start, each part is rated in each of four categories:

- x: Extremely cool looking
- m: Musical (it makes a noise when you hit it)
- a: Aerodynamic
- s: Shiny
Then, each part is sent through a series of workflows that will ultimately accept or reject the part. Each workflow has a name and contains a list of rules; each rule specifies a condition and where to send the part if the condition is true. The first rule that matches the part being considered is applied immediately, and the part moves on to the destination described by the rule. (The last rule in each workflow has no condition and always applies if reached.)

Consider the workflow ex{x>10:one,m<20:two,a>30:R,A}. This workflow is named ex and contains four rules. If workflow ex were considering a specific part, it would perform the following steps in order:

- Rule "x>10:one": If the part's x is more than 10, send the part to the workflow named one.
- Rule "m<20:two": Otherwise, if the part's m is less than 20, send the part to the workflow named two.
- Rule "a>30:R": Otherwise, if the part's a is more than 30, the part is immediately rejected (R).
- Rule "A": Otherwise, because no other rules matched the part, the part is immediately accepted (A).
If a part is sent to another workflow, it immediately switches to the start of that workflow instead and never returns. If a part is accepted (sent to A) or rejected (sent to R), the part immediately stops any further processing.

The system works, but it's not keeping up with the torrent of weird metal shapes. The Elves ask if you can help sort a few parts and give you the list of workflows and some part ratings (your puzzle input). For example:
```
px{a<2006:qkq,m>2090:A,rfg}
pv{a>1716:R,A}
lnx{m>1548:A,A}
rfg{s<537:gd,x>2440:R,A}
qs{s>3448:A,lnx}
qkq{x<1416:A,crn}
crn{x>2662:A,R}
in{s<1351:px,qqz}
qqz{s>2770:qs,m<1801:hdj,R}
gd{a>3333:R,R}
hdj{m>838:A,pv}

{x=787,m=2655,a=1222,s=2876}
{x=1679,m=44,a=2067,s=496}
{x=2036,m=264,a=79,s=2244}
{x=2461,m=1339,a=466,s=291}
{x=2127,m=1623,a=2188,s=1013}
```
The workflows are listed first, followed by a blank line, then the ratings of the parts the Elves would like you to sort. All parts begin in the workflow named in. In this example, the five listed parts go through the following workflows:

- {x=787,m=2655,a=1222,s=2876}: in -> qqz -> qs -> lnx -> A
- {x=1679,m=44,a=2067,s=496}: in -> px -> rfg -> gd -> R
- {x=2036,m=264,a=79,s=2244}: in -> qqz -> hdj -> pv -> A
- {x=2461,m=1339,a=466,s=291}: in -> px -> qkq -> crn -> R
- {x=2127,m=1623,a=2188,s=1013}: in -> px -> rfg -> A
Ultimately, three parts are accepted. Adding up the x, m, a, and s rating for each of the accepted parts gives 7540 for the part with x=787, 4623 for the part with x=2036, and 6951 for the part with x=2127. Adding all of the ratings for all of the accepted parts gives the sum total of 19114.

Sort through all of the parts you've been given; what do you get if you add together all of the rating numbers for all of the parts that ultimately get accepted?

In [30]:
from rich import print as rprint
from dataclasses import dataclass


@dataclass
class Rating:
    x: int
    m: int
    a: int
    s: int

    @property
    def score(self) -> int:
        return self.x + self.m + self.a + self.s


def parse_workflows(workflows) -> dict:
    workflow_rules = {}
    for workflow in workflows:
        workflow_name, workflow_rules_str = workflow.split("{")
        workflow_rules_str = workflow_rules_str[:-1]
        workflow_rules[workflow_name] = workflow_rules_str.split(",")
    return workflow_rules


def run_workflow(rating: Rating, workflows: dict, workflow_name: str = None):
    workflow_name = workflow_name or "in"
    if workflow_name == "R":
        return 0
    if workflow_name == "A":
        return rating.score

    workflow = workflows[workflow_name]
    # {"tq": ["x<2961:A", "m<1415:A", "A"]}
    # rprint(f"{workflow_name=} -> {workflow}")
    for i, rule in enumerate(workflow):
        if ":" in rule:
            rule, next_workflow_name = rule.split(":")
            eval_str = f"rating.{rule}"
            if eval(eval_str):
                # rprint(f"\t{eval_str}=true, running workflow `{next_workflow_name}`")

                return run_workflow(rating, workflows, next_workflow_name)
            else:
                other_rule = ""
                if i != len(workflow) - 1:
                    other_rule = f", continuing to `{workflow[i + 1]}`"
                # rprint(f"\t{eval_str}=false, skipping workflow `{next_workflow_name}`{other_rule}")
                continue
        else:
            return run_workflow(rating, workflows, rule)


def run_workflow_rules(workflows: list[str], ratings: list[str]):
    workflow_rules: dict = parse_workflows(workflows)
    # rprint(workflow_rules)
    score = 0
    for rating in ratings:
        rating_obj = Rating(*[int(value.split("=")[1]) for value in rating.strip("{}").split(",")])
        # rprint(rating_obj)
        result = run_workflow(rating_obj, workflow_rules)
        # rprint(f"score={result}")
        score += result
    return score


test_input = """
px{a<2006:qkq,m>2090:A,rfg}
pv{a>1716:R,A}
lnx{m>1548:A,A}
rfg{s<537:gd,x>2440:R,A}
qs{s>3448:A,lnx}
qkq{x<1416:A,crn}
crn{x>2662:A,R}
in{s<1351:px,qqz}
qqz{s>2770:qs,m<1801:hdj,R}
gd{a>3333:R,R}
hdj{m>838:A,pv}

{x=787,m=2655,a=1222,s=2876}
{x=1679,m=44,a=2067,s=496}
{x=2036,m=264,a=79,s=2244}
{x=2461,m=1339,a=466,s=291}
{x=2127,m=1623,a=2188,s=1013}""".strip()

test_workflows, test_ratings = test_input.split("\n\n")
test_result = run_workflow_rules(
    test_workflows.split("\n"),
    test_ratings.split("\n"),
)
test_result

19114

In [31]:
input_str = open("../inputs/19.txt").read()
workflows, ratings = input_str.split("\n\n")
result = run_workflow_rules(
    workflows.split("\n"),
    ratings.split("\n"),
)
result

287054

## Part 2

Even with your help, the sorting process still isn't fast enough.

One of the Elves comes up with a new plan: rather than sort parts individually through all of these workflows, maybe you can figure out in advance which combinations of ratings will be accepted or rejected.

Each of the four ratings (x, m, a, s) can have an integer value ranging from a minimum of 1 to a maximum of 4000. Of all possible distinct combinations of ratings, your job is to figure out which ones will be accepted.

In the above example, there are 167409079868000 distinct combinations of ratings that will be accepted.

Consider only your list of workflows; the list of part ratings that the Elves wanted you to sort is no longer relevant. How many distinct combinations of ratings will be accepted by the Elves' workflows?

In [36]:
import itertools
from tqdm.auto import tqdm


def generate_ratings():
    all_possible_ratings = itertools.product(range(1, 4001), repeat=4)
    for rating in all_possible_ratings:
        yield Rating(*rating)


def run_workflow_rules2(workflows: list[str]):
    workflow_rules: dict = parse_workflows(workflows)
    # rprint(workflow_rules)
    score = 0
    for rating_obj in tqdm(generate_ratings()):
        # rprint(rating_obj)
        result = run_workflow(rating_obj, workflow_rules)
        # rprint(f"score={result}")
        score += result
    return score


test_workflows = test_input.split("\n\n")[0]
test_result = run_workflow_rules2(test_workflows.split("\n"))
test_result

11931822it [03:27, 57464.86it/s]


KeyboardInterrupt: 